# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [109]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. '''

## 2. Ask for parameters

In [136]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [137]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

* **xij:** Liters of component *i* used in the production of product *j*, where *i* = {C1, C2} and *j* = {GCA, GCB, GCC}. 

This gives us the following six variables:

* **xC1_GCA:** Liters of C1 used in GCA
* **xC2_GCA:** Liters of C2 used in GCA
* **xC1_GCB:** Liters of C1 used in GCB
* **xC2_GCB:** Liters of C2 used in GCB
* **xC1_GCC:** Liters of C1 used in GCC
* **xC2_GCC:** Liters of C2 used in GCC 


# 2. Ask for objective

In [138]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [139]:
Markdown(response2.text)

```
Maximize: 120(xC1_GCA + xC2_GCA) + 135(xC1_GCB + xC2_GCB) + 155(xC1_GCC + xC2_GCC) 
``` 


# 3. Ask for constraints

In [140]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [141]:
Markdown(response3.text)

## Constraints:

**Resource Constraints:**

* **Component Availability:**
    * xC1_GCA + xC1_GCB + xC1_GCC ≤ 10000 (C1 availability)
    * xC2_GCA + xC2_GCB + xC2_GCC ≤ 15000 (C2 availability) 

* **Demand Constraints:**
    * xC1_GCA + xC2_GCA ≥ 6000 (Minimum demand for GCA)
    * xC1_GCB + xC2_GCB ≥ 7000 (Minimum demand for GCB)
    * xC1_GCC + xC2_GCC ≥ 9000 (Minimum demand for GCC)

**Critical Element Constraints:**

* **GCA (at least 0.3 fraction of critical element):**
    * (0.4 * xC1_GCA + 0.2 * xC2_GCA) / (xC1_GCA + xC2_GCA)  ≥ 0.3 

* **GCB (at most 0.3 fraction of critical element):** 
    * (0.4 * xC1_GCB + 0.2 * xC2_GCB) / (xC1_GCB + xC2_GCB) ≤ 0.3

* **GCC (minimum ratio of C1 to C2 of 0.3):**
    * xC1_GCC  ≥ 0.3 * xC2_GCC 

**Non-negativity Constraints:**

* xC1_GCA ≥ 0 
* xC2_GCA ≥ 0
* xC1_GCB ≥ 0
* xC2_GCB ≥ 0
* xC1_GCC ≥ 0
* xC2_GCC ≥ 0 


## 3. Generate the pyomo code

In [142]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [143]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets (optional but improves readability)
components = ['C1', 'C2']
products = ['GCA', 'GCB', 'GCC']

# Define variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Objective function
def objective_rule(model):
    return sum(120*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']) +
               135*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']) + 
               155*(model.x['C1', 'GCC'] + model.x['C2', 'GCC']) 
               for i in components for j in products) 
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
model.constraints = pyo.ConstraintList()

# Resource Constraints
model.constraints.add(sum(model.x['C1', j] for j in products) <= 10000) # C1 availability
model.constraints.add(sum(model.x['C2', j] for j in products) <= 15000) # C2 availability

# Demand Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.x['C2', 'GCA'] >= 6000) 
model.constraints.add(model.x['C1', 'GCB'] + model.x['C2', 'GCB'] >= 7000)
model.constraints.add(model.x['C1', 'GCC'] + model.x['C2', 'GCC'] >= 9000) 

# Critical Element Constraints
model.constraints.add((0.4 * model.x['C1', 'GCA'] + 0.2 * model.x['C2', 'GCA']) >= 
                      0.3 * (model.x['C1', 'GCA'] + model.x['C2', 'GCA']))
model.constraints.add((0.4 * model.x['C1', 'GCB'] + 0.2 * model.x['C2', 'GCB']) <= 
                      0.3 * (model.x['C1', 'GCB'] + model.x['C2', 'GCB']))
model.constraints.add(model.x['C1', 'GCC'] >= 0.3 * model.x['C2', 'GCC'])

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver (e.g., 'glpk', 'gurobi')
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"x[{i}, {j}] = {pyo.value(model.x[i, j])}")

print(f"Total Profit = {pyo.value(model.profit)}") 
``` 


## 4. Input problem data and try running the generated code

In [146]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets (optional but improves readability)
components = ['C1', 'C2']
products = ['GCA', 'GCB', 'GCC']

# Define variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Objective function
def objective_rule(model):
    return sum(120*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']) +
               135*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']) + 
               155*(model.x['C1', 'GCC'] + model.x['C2', 'GCC']) 
               for i in components for j in products) 
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
model.constraints = pyo.ConstraintList()

# Resource Constraints
model.constraints.add(sum(model.x['C1', j] for j in products) <= 10000) # C1 availability
model.constraints.add(sum(model.x['C2', j] for j in products) <= 15000) # C2 availability

# Demand Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.x['C2', 'GCA'] >= 6000) 
model.constraints.add(model.x['C1', 'GCB'] + model.x['C2', 'GCB'] >= 7000)
model.constraints.add(model.x['C1', 'GCC'] + model.x['C2', 'GCC'] >= 9000) 

# Critical Element Constraints
model.constraints.add((0.4 * model.x['C1', 'GCA'] + 0.2 * model.x['C2', 'GCA']) >= 
                      0.3 * (model.x['C1', 'GCA'] + model.x['C2', 'GCA']))
model.constraints.add((0.4 * model.x['C1', 'GCB'] + 0.2 * model.x['C2', 'GCB']) <= 
                      0.3 * (model.x['C1', 'GCB'] + model.x['C2', 'GCB']))
model.constraints.add(model.x['C1', 'GCC'] >= 0.3 * model.x['C2', 'GCC'])

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver (e.g., 'glpk', 'gurobi')
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"x[{i}, {j}] = {pyo.value(model.x[i, j])}")

print(f"Total Profit = {pyo.value(model.profit)}") 

Optimal Solution:
x[C1, GCA] = 3730.76923076923
x[C1, GCB] = 3500.0
x[C1, GCC] = 2769.23076923077
x[C2, GCA] = 2269.23076923077
x[C2, GCB] = 3500.0
x[C2, GCC] = 9230.76923076923
Total Profit = 21150000.0


## 5. Correct the code to verify model viability (optional)

In [151]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets (optional but improves readability)
components = ['C1', 'C2']
products = ['GCA', 'GCB', 'GCC']

# Define variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Objective function
def objective_rule(model):
    p = 120*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']) + 135*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']) + 155*(model.x['C1', 'GCC'] + model.x['C2', 'GCC'])
    return p
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
model.constraints = pyo.ConstraintList()

# Resource Constraints
model.constraints.add(sum(model.x['C1', j] for j in products) <= 10000) # C1 availability
model.constraints.add(sum(model.x['C2', j] for j in products) <= 15000) # C2 availability

# Demand Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.x['C2', 'GCA'] >= 6000) 
model.constraints.add(model.x['C1', 'GCB'] + model.x['C2', 'GCB'] >= 7000)
model.constraints.add(model.x['C1', 'GCC'] + model.x['C2', 'GCC'] >= 9000) 

# Critical Element Constraints
model.constraints.add((0.4 * model.x['C1', 'GCA'] + 0.2 * model.x['C2', 'GCA']) >= 
                      0.3 * (model.x['C1', 'GCA'] + model.x['C2', 'GCA']))
model.constraints.add((0.4 * model.x['C1', 'GCB'] + 0.2 * model.x['C2', 'GCB']) <= 
                      0.3 * (model.x['C1', 'GCB'] + model.x['C2', 'GCB']))
model.constraints.add(model.x['C1', 'GCC'] >= 0.3 * model.x['C2', 'GCC'])

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver (e.g., 'glpk', 'gurobi')
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"x[{i}, {j}] = {pyo.value(model.x[i, j])}")

print(f"Total Profit = {pyo.value(model.profit)}") 

Optimal Solution:
x[C1, GCA] = 3730.76923076923
x[C1, GCB] = 3500.0
x[C1, GCC] = 2769.23076923077
x[C2, GCA] = 2269.23076923077
x[C2, GCB] = 3500.0
x[C2, GCC] = 9230.76923076923
Total Profit = 3525000.0


## 6. Print the responses

In [156]:
print(response.text)

## Variables:

* **xij:** Liters of component *i* used in the production of product *j*, where *i* = {C1, C2} and *j* = {GCA, GCB, GCC}. 

This gives us the following six variables:

* **xC1_GCA:** Liters of C1 used in GCA
* **xC2_GCA:** Liters of C2 used in GCA
* **xC1_GCB:** Liters of C1 used in GCB
* **xC2_GCB:** Liters of C2 used in GCB
* **xC1_GCC:** Liters of C1 used in GCC
* **xC2_GCC:** Liters of C2 used in GCC 



In [157]:
print(response2.text)

```
Maximize: 120(xC1_GCA + xC2_GCA) + 135(xC1_GCB + xC2_GCB) + 155(xC1_GCC + xC2_GCC) 
``` 



In [159]:
print(response3.text)

## Constraints:

**Resource Constraints:**

* **Component Availability:**
    * xC1_GCA + xC1_GCB + xC1_GCC ≤ 10000 (C1 availability)
    * xC2_GCA + xC2_GCB + xC2_GCC ≤ 15000 (C2 availability) 

* **Demand Constraints:**
    * xC1_GCA + xC2_GCA ≥ 6000 (Minimum demand for GCA)
    * xC1_GCB + xC2_GCB ≥ 7000 (Minimum demand for GCB)
    * xC1_GCC + xC2_GCC ≥ 9000 (Minimum demand for GCC)

**Critical Element Constraints:**

* **GCA (at least 0.3 fraction of critical element):**
    * (0.4 * xC1_GCA + 0.2 * xC2_GCA) / (xC1_GCA + xC2_GCA)  ≥ 0.3 

* **GCB (at most 0.3 fraction of critical element):** 
    * (0.4 * xC1_GCB + 0.2 * xC2_GCB) / (xC1_GCB + xC2_GCB) ≤ 0.3

* **GCC (minimum ratio of C1 to C2 of 0.3):**
    * xC1_GCC  ≥ 0.3 * xC2_GCC 

**Non-negativity Constraints:**

* xC1_GCA ≥ 0 
* xC2_GCA ≥ 0
* xC1_GCB ≥ 0
* xC2_GCB ≥ 0
* xC1_GCC ≥ 0
* xC2_GCC ≥ 0 



In [158]:
print(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define sets (optional but improves readability)
components = ['C1', 'C2']
products = ['GCA', 'GCB', 'GCC']

# Define variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Objective function
def objective_rule(model):
    return sum(120*(model.x['C1', 'GCA'] + model.x['C2', 'GCA']) +
               135*(model.x['C1', 'GCB'] + model.x['C2', 'GCB']) + 
               155*(model.x['C1', 'GCC'] + model.x['C2', 'GCC']) 
               for i in components for j in products) 
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
model.constraints = pyo.ConstraintList()

# Resource Constraints
model.constraints.add(sum(model.x['C1', j] for j in products) <= 10000) # C1 availability
model.constraints.add(sum(model.x['C2', j] for j in products) <= 15000) # C2 availability

# Demand Constraints
model.constraints.add(model.x['C1', 'GCA'] + model.